In [3]:
import numpy as np
import matplotlib.pyplot as plt
import stim
from lib.stabilizer import measurement_gadgets, StabilizerCode, stabilizer_circuits
from lib.color_compass import Lattice2D, compass_to_surface
from lib.decoder import checkmatrix,pL_from_checkmatrix
from lib.stim2pymatching import estimate_pL_noisy_graph
import stimcirq
from cirq.contrib.svg import SVGCircuit

The basic learning loop is as follows 

0) Fix a hidden Pauli noise model for the gates and measurements
1) Pick a compass code and get its stabilizers and logical operators
2) Construct a noisy stim circuit for the code under the noise model 
3) Simulate and get logical error rate
4) Go to 1) and repeat

In [4]:
"""noise model construction""" 

class NoisyGateSet():
    """
    Constructs noisy stim circuits from stabilizers and logical operations
    
    A noise model takes an operation and outputs a noisy gadget 
    """
    def __init__(self):
        self.gate_1qb_rates = [0, 0, 0]
        self.gate_2qb_rates = [0]*15
        self.meas_error_rate = 0
        
    def set_1qb_rates(self, rates):
        self.gate_1qb_rates = rates.copy()
    
    def set_2qb_rates(self, rates):
        self.gate_2qb_rates = rates.copy()
        
    def set_meas_error(self, rate):
        self.meas_error_rate = rate
        
    def noise_string_1qb(self):
        """returns stim string for 1qb Pauli channel"""
        noise_string = 'PAULI_CHANNEL_1('
        for r in self.gate_1qb_rates:
            noise_string += f'{r},'
        noise_string = noise_string[:len(noise_string)-1] # remove last comma
        noise_string += ') '
        return noise_string
    
    def noise_string_2qb(self):
        """returns stim string for 2qb Pauli channel"""
        noise_string = 'PAULI_CHANNEL_2('
        for r in self.gate_2qb_rates:
            noise_string += f'{r},'
        noise_string = noise_string[:len(noise_string)-1] # remove last comma
        noise_string += ') '
        return noise_string
    
    def measurement_gadget(self, pauli_observable):
        """Gadget to directly measure the observable with measurement noise"""
        circ_string = ''
        xpart = ''
        ypart = ''
        zpart = ''
        for i, pauli in enumerate(pauli_observable):
            if pauli == 'X':
                xpart += f'{i} '
            elif pauli == 'Y':
                ypart += f'{i} '
            elif pauli == 'Z':
                zpart += f'{i} '
        if zpart != '':
            circ_string += f'MZ({self.meas_error_rate}) ' + zpart + '\n'
        if xpart != '':
            circ_string = f'MX({self.meas_error_rate}) ' + xpart + '\n'
        if ypart != '':
            circ_string += f'MY({self.meas_error_rate}) ' + ypart + '\n'
        return stim.Circuit(circ_string)
        

    def stabilizer_gadget(self, stabilizer_in, ancilla_index, construction='cnot'):
        """
        Input:
            stabilizer: a single stabilizer written in terms of {I/_,X,Y,Z}
            construction: direct or hadamard:
                1) `cnot` using only CNOTs from data to ancilla along with single qubit gates
                    - H then S    : rotates Z basis -> Y basis
                    - S_dag then H: rotates Y basis -> Z basis
                    verifiable via checking that: Y stabilizer == kron(S@H,I) @ CNOT @ kron(H@S_dag)
                2) `hadamard` using H gates on ancilla and C-Pauli from ancilla to data
        Output:
            Measurement gadget

        the data qubits that are indicated in the stabilizer appear first
        the ancilla index starts from 0, which is the first ancilla qubit after the data 
        """

        # allow both '_' and 'I' in stabilizers
        stabilizer = stabilizer_in.replace('_','I')
        
        N = len(stabilizer)
        circ_string = ''
        if construction == 'cnot':
            for i, pauli in enumerate(stabilizer):
                if pauli == 'Z':
                    # Z-gates are just cnots from data to ancilla
                    noise_string = self.noise_string_2qb() + f'{i} {N+ancilla_index}\n'
                    circ_string += f'CX {i} {ancilla_index+N} \n' 
                    circ_string += noise_string
                elif pauli == 'X':
                    # X-gates are conjugated by hadamards
                    noise_string_1qb = self.noise_string_1qb() + f'{i}\n'
                    noise_string_2qb = self.noise_string_2qb() + f'{i} {N+ancilla_index}\n'
                    circ_string += f'H {i} \n'
                    circ_string += noise_string_1qb
                    circ_string += f'CX {i} {ancilla_index+N} \n' 
                    circ_string += noise_string_2qb
                    circ_string += f'H {i} \n'
                    circ_string += noise_string_1qb
                elif pauli == 'Y':
                    # Y-gates are conjugated by S-gates and hadamards
                    noise_string_1qb = self.noise_string_1qb() + f'{i}\n'
                    noise_string_2qb = self.noise_string_2qb() + f'{i} {N+ancilla_index}\n'
                    circ_string = f'S_DAG {i} \n'
                    circ_string += noise_string_1qb
                    circ_string += f'H {i} \n'
                    circ_string += noise_string_1qb
                    circ_string += f'CX {i} {ancilla_index+N} \n' 
                    circ_string += noise_string_2qb
                    circ_string += f'H {i} \n'
                    circ_string += noise_string_1qb
                    circ_string += f'S {i} \n'
                    circ_string += noise_string_1qb 

            # noisy ancilla measurement
            circ_string += f'MR({self.meas_error_rate}) {N+ancilla_index}\n'
        return stim.Circuit(circ_string)
    
    def stabilizer_gadget_v2(self, stabilizer_in, ancilla_index, construction='cnot'):
        """
        Input:
            stabilizer: a single stabilizer written in terms of {I/_,X,Y,Z}
            construction: direct or hadamard:
                1) `cnot` using only CNOTs from data to ancilla along with single qubit gates
                    - H then S    : rotates Z basis -> Y basis
                    - S_dag then H: rotates Y basis -> Z basis
                    verifiable via checking that: Y stabilizer == kron(S@H,I) @ CNOT @ kron(H@S_dag)
                2) `hadamard` using H gates on ancilla and C-Pauli from ancilla to data
        Output:
            Measurement gadget

        the data qubits that are indicated in the stabilizer appear first
        the ancilla index starts from 0, which is the first ancilla qubit after the data 
        """

        # allow both '_' and 'I' in stabilizers
        stabilizer = stabilizer_in.replace('_','I')
        
        N = len(stabilizer)
        circ_string = f'MPP({self.meas_error_rate}) '
        for i, pauli in enumerate(stabilizer):
            if (pauli != 'I'):
                circ_string += f'{pauli}{i}*'
        circ_string = circ_string[:-1] + '\n'
        return stim.Circuit(circ_string)
                
                
#                 bS, bH, mid, aH, aS = '','','','','' # before Sgate, before Hgate, mid gate, after Hgate, after Sgate
#                 for i, pauli in enumerate(stab):
#                     s = f'{i} '
#                     if pauli != 'I':
#                         mid += s + f'{ancilla+N} '
#                         if pauli == 'X':
#                             bH += s
#                             aH += s
#                         elif pauli == 'Y':
#                             bS += s
#                             bH += s
#                             aH += s
#                             aS += s
#                 # gate noise
#                 if gate_noise1:
#                     bS = f'S_DAG {bS} \n{gate_noise1} {bS}' if len(bS) > 0 else ''
#                     bH = f'H {bH}     \n{gate_noise1} {bH}' if len(bH) > 0 else ''
#                     aH = f'H {aH}     \n{gate_noise1} {aH}' if len(aH) > 0 else ''
#                     aS = f'S {aS}     \n{gate_noise1} {aS}' if len(aS) > 0 else ''
#                 else:
#                     bS = f'S_DAG {bS}' if len(bS) > 0 else ''
#                     bH = f'H {bH}    ' if len(bH) > 0 else ''
#                     aH = f'H {aH}    ' if len(aH) > 0 else ''
#                     aS = f'S {aS}    ' if len(aS) > 0 else ''

#                 if gate_noise2:
#                     mid = f'CX {mid} \n{gate_noise2} {mid}' if len(mid) > 0 else ''
#                 else:
#                     mid = f'CX {mid}' if len(mid) > 0 else ''

#                 # measurement noise
#                 if meas_noise:
#                     aS += f'\n{meas_noise} {ancilla + N}'
#                 aS += f'\nMR {ancilla + N}'
#                 gadgets.append(stim.Circuit('\n'.join([bS,bH,mid,aH,aS])))

#             elif construction == 'hadamard':
#                 before, mX, mY, mZ, after = '','','','',''
#                 s1 = f'{ancilla+N} '
#                 before += s1
#                 for i, pauli in enumerate(stab):
#                     s2 = f'{ancilla+N} {i} '
#                     if pauli == 'X': 
#                         mX += s2
#                     elif pauli == 'Y': 
#                         mY += s2
#                     elif pauli == 'Z': 
#                         mZ += s2
#                 after += s1 

#                 # gate noise
#                 if gate_noise1:
#                     before = f'H {before} \n{gate_noise1} {before}' if len(before) > 0 else ''
#                     after  = f'H {after}  \n{gate_noise1} {after}' if len(after) > 0 else ''
#                 else:
#                     before = f'H {before}' if len(before) > 0 else ''
#                     after  = f'H {after}' if len(after) > 0 else ''

#                 if gate_noise2:
#                     mX = f'CX {mX} \n{gate_noise2} {mX}' if len(mX) > 0 else ''
#                     mY = f'CY {mY} \n{gate_noise2} {mY}' if len(mY) > 0 else ''
#                     mZ = f'CZ {mZ} \n{gate_noise2} {mZ}' if len(mZ) > 0 else ''
#                 else:
#                     mX = f'CX {mX}' if len(mX) > 0 else ''
#                     mY = f'CY {mY}' if len(mY) > 0 else ''
#                     mZ = f'CZ {mZ}' if len(mZ) > 0 else ''

#                 # measurement noise
#                 if meas_noise:
#                     after += f'\n{meas_noise} {ancilla + N}'
#                 after += f'\nMR {ancilla + N}'
#                 gadgets.append(stim.Circuit('\n'.join([before,mX,mY,mZ,after])))

 #       return gadgets
        
        
        
    

In [5]:
"""pick the compass code"""
dim = 9
lat = Lattice2D(dim, dim)

coloring = np.random.randint(-1, 2, size=(dim-1)**2)
lat.color_lattice(coloring)

In [6]:
print(lat)

000---001---002---003---004---005---006---007---008
 |  ░  |     |     |  ░  |  ░  |     |     |  ▓  |
009---010---011---012---013---014---015---016---017
 |  ░  |  ▓  |     |  ▓  |  ░  |  ▓  |  ▓  |  ░  |
018---019---020---021---022---023---024---025---026
 |  ▓  |     |  ▓  |  ▓  |  ░  |     |     |     |
027---028---029---030---031---032---033---034---035
 |  ▓  |     |  ░  |     |  ▓  |     |     |  ▓  |
036---037---038---039---040---041---042---043---044
 |     |     |     |     |  ▓  |  ▓  |  ░  |  ▓  |
045---046---047---048---049---050---051---052---053
 |  ░  |  ▓  |     |  ░  |     |  ▓  |  ░  |  ▓  |
054---055---056---057---058---059---060---061---062
 |  ░  |  ▓  |     |  ░  |  ░  |  ░  |  ░  |  ▓  |
063---064---065---066---067---068---069---070---071
 |     |     |  ▓  |     |  ░  |     |  ▓  |  ░  |
072---073---074---075---076---077---078---079---080



In [7]:
"""construct the encoding circuit"""
encoding = StabilizerCode(lat.getS()).encoding_circuit(stim=True)

In [8]:
"""fix a gate set"""
gs = NoisyGateSet()
# gs.set_1qb_rates([0.0004, 0.001, 0.001]) # With circuit-level noise
gs.set_1qb_rates([0,0,0]) # Without circuit-level noise
gs.set_meas_error(0.001)

In [9]:
"""make a stabilizer gadgets"""
stab_gadget = gs.stabilizer_gadget_v2(lat.getS()[0], 0)
print(stab_gadget)

MPP(0.001) X0*X1*X2*X3*X4*X5*X6*X7*X9*X10*X11*X12*X13*X14*X15*X16


In [10]:
"""make the observable measurement"""
meas_gadget = gs.measurement_gadget(lat.Lx)
print(meas_gadget)

MX(0.001) 0 1 2 3 4 5 6 7 8


In [11]:
def compile_compass_circuit(lattice, noisy_gateset, rounds):
    """
    compiles a compass code lattice into a stim circuits with detectors
    
    Input:
        lattice: a compass code lattice
        noisy_gateset: noisy model for the processors
    Output:
        stim circuit: detectors can be sampled for syndromes 
        
    Detector convention: 
        
    """

    # [0] encoding
    circ = StabilizerCode(lattice.getS()).encoding_circuit(stim=True)

    # [1] stabilizer gadgets           
    num_stabs = len(lattice.getS())
    # first round
    for idx, sx in enumerate(lattice.getSx()):
        circ += gs.stabilizer_gadget(sx, idx)
    for idx, sx in enumerate(lattice.getSx()):
        circ += gs.stabilizer_gadget(sx, idx)
        circ += stim.Circuit(f'DETECTOR({idx}, 0, 0) rec[-{idx+1}]')
    # subsequent rounds
    for r in range(rounds - 1):
        for idx, sx in enumerate(lattice.getSx()):
            circ += gs.stabilizer_gadget(sx, idx)
            circ += stim.Circuit(f'DETECTOR({idx+1}, 0, {r+1}) rec[-{idx+1}] rec[{-idx-num_stabs}]')
    # [2] logical measurement 

    return circ

def compile_compass_circuit_v2(lattice, noisy_gateset, rounds):
    """
    compiles a compass code lattice into a stim circuits with detectors
    
    Input:
        lattice: a compass code lattice
        noisy_gateset: noisy model for the processors
    Output:
        stim circuit: detectors can be sampled for syndromes 
        
    Detector convention: 
        
    """

    # [0] encoding
    circ = StabilizerCode(lattice.getS()).encoding_circuit(stim=True)

    # [1] stabilizer gadgets           
    num_stabs = len(lattice.getS())
    # first round
    for idx, sx in enumerate(lattice.getSx()):
        circ += gs.stabilizer_gadget_v2(sx, idx)
    for idx, sx in enumerate(lattice.getSx()):
        circ += gs.stabilizer_gadget_v2(sx, idx)
        # circ += stim.Circuit(f'DETECTOR({idx}, 0, 0) rec[-{idx+1}]')
        circ += stim.Circuit(f'DETECTOR rec[-]')
    # subsequent rounds
    for r in range(rounds - 1):
        for idx, sx in enumerate(lattice.getSx()):
            circ += gs.stabilizer_gadget_v2(sx, idx)
            # circ += stim.Circuit(f'DETECTOR({idx+1}, 0, {r+1}) rec[-{idx+1}] rec[{-idx-num_stabs}]')
    # [2] logical measurement 

    return circ
        
    

In [13]:
circ = compile_compass_circuit(lat, gs, 3)
print(circ)

H 0 8 9 11 13 15 16 18 19 21 22 27 28 32 35 36 41 42 44 45 47 51 53 54 56 62 63 66 70
TICK
CX 0 1 9 10 13 14 18 37 22 23 28 29 45 65 51 52 54 55 56 57 63 64 66 67
TICK
CX 0 4 9 50 18 38 19 37 45 46 66 68
TICK
CX 0 5 9 60 13 50 18 39 19 38 27 37 66 69
TICK
CX 0 17 13 60 18 40 19 39 27 38 28 37
TICK
CX 0 6 8 17 13 26 18 50 19 40 27 39 28 38 36 37
TICK
CX 0 75 13 24 16 17 18 60 22 50 27 40 28 39 36 38
TICK
CX 0 76 9 75 15 24 16 26 22 60 27 50 28 40 36 39
TICK
CX 0 79 9 76 18 75 22 26 27 60 32 50 36 40
TICK
CX 0 7 9 79 18 76 22 24 27 75 32 34 36 50
TICK
CX 0 3 9 30 18 79 27 76 32 60 41 50
TICK
CX 0 2 9 20 11 30 27 79 36 60 47 50
TICK
CX 0 72 9 61 11 20 19 30 36 75 42 60
TICK
CX 0 73 9 31 11 12 13 61 19 20 21 30 36 76 47 60
TICK
CX 0 74 9 71 13 25 18 61 28 30 36 79 47 75 51 60
TICK
CX 0 77 9 72 13 31 16 25 22 61 47 76 56 60
TICK
CX 0 78 9 73 13 71 19 31 22 25 27 61 47 79 56 75
TICK
CX 0 80 9 74 18 71 22 31 32 61 56 76 66 75
TICK
CX 9 77 18 72 22 71 28 31 32 33 36 61 56 79 66 76
TICK
CX 9 78

In [14]:
sampler = circ.compile_detector_sampler()
print(sampler.sample(shots=2))

[[False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False]
 [False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False Fal